# Combined fits to alpha, epsilon and beta for second generation mocks 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
# read in the chains for the fits
# lets first just look at alpha and epsilon
# neglecting low z ELGs as they are only weakly constraining and likewise QSO cross-correlations as they are likely small

# using post-recon mocks for all except QSOs - too few QSOs to have good reconstruction 
path = '/global/u1/a/abbew25/files_secondgen_chains/'

listt = [r'$\alpha$', r'$\epsilon$', 'weights']
#chains_ELGs_z_0p8_1p1 = pd.read_csv(path + 'ELG_08_11_recon' + '.csv')[list]
chains_ELGs_z_1p1_1p6 = pd.read_csv(path + 'ELG_11_16_recon' + '.csv')[listt]
chains_LRGs_z_0p4_0p6 = pd.read_csv(path + 'LRG_04_06_recon' + '.csv')[listt]
chains_LRGs_z_0p6_0p8 = pd.read_csv(path + 'LRG_06_08_recon' + '.csv')[listt]
chains_LRGs_z_0p8_1p1 = pd.read_csv(path + 'LRG_08_11_recon' + '.csv')[listt]
chains_QSOs_z_0p8_2p1 = pd.read_csv(path + 'QSO_08_21_prerecon' + '.csv')[listt]

;

In [ ]:
print(chains_ELGs_z_1p1_1p6)

In [ ]:
# now interpolate the arrays with something called a Gaussian kernel density estimate (KDE)
from scipy.stats import gaussian_kde

In [ ]:
kde_elg_11_16 = gaussian_kde(np.vstack([
    chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy(),
    chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy()
    ]), 
    weights=chains_ELGs_z_1p1_1p6['weights'].to_numpy())

kde_lrg_04_06 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p4_0p6[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p4_0p6[r'$\epsilon$'].to_numpy()
    ]), 
    weights=chains_LRGs_z_0p4_0p6['weights'].to_numpy())

kde_lrg_06_08 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p6_0p8[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p6_0p8[r'$\epsilon$'].to_numpy()
    ]), 
    weights=chains_LRGs_z_0p6_0p8['weights'].to_numpy())

kde_lrg_08_11 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p8_1p1[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p8_1p1[r'$\epsilon$'].to_numpy()
    ]), 
    weights=chains_LRGs_z_0p8_1p1['weights'].to_numpy())

kde_qso_08_21 = gaussian_kde(np.vstack([
    chains_QSOs_z_0p8_2p1[r'$\alpha$'].to_numpy(),
    chains_QSOs_z_0p8_2p1[r'$\epsilon$'].to_numpy()
    ]), 
    weights=chains_QSOs_z_0p8_2p1['weights'].to_numpy())

In [ ]:
xmin = np.min(chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy())
xmax = np.max(chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy())
ymin = np.min(chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy())
ymax = np.max(chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy())

x, y = np.linspace(xmin, xmax, 100), np.linspace(ymin, ymax, 100)
X, Y = np.meshgrid(x, y) # get a 2D grid of points 

positions = np.vstack([X.ravel(), Y.ravel()]) # unravel the points to have a 2 1D arrays 

# use those points to interpolate
values = np.vstack([chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy(), chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy()])

# take the interpolated points and shape the output to be 2D 
z0 = kde_elg_11_16(positions)
Z = np.reshape(z0, X.shape)
Z = np.flip(Z, axis=0)

In [ ]:
# plot the results 
import matplotlib as mpl

fig, ax = plt.subplots()

# ax.plot(chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy(), chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy(), 
#     'k.', markersize=0.5, alpha = 0.2)

maxxarg, maxyarg = np.unravel_index(np.argmax(np.flip(Z,axis=0).T), shape=X.shape)

plt.scatter(x[maxxarg], y[maxyarg], color='red', marker='*', s=26)

ax.imshow(Z, cmap=plt.cm.gist_earth_r, extent=[xmin, xmax, ymin, ymax])

norm = mpl.colors.Normalize(vmin=np.min(Z.reshape(-1)), vmax=np.max(Z.reshape(-1)))

sm = plt.cm.ScalarMappable(cmap=plt.cm.gist_earth_r, norm=norm)
plt.colorbar(sm, ax=ax)

ax.set_xlim([xmin, xmax])

ax.set_ylim([ymin, ymax])

plt.show()


In [ ]:
# ok, now lets add the 'likelihoods' from the functions together by taking the log of all of them then summing them 

kdes_list = [kde_elg_11_16, kde_lrg_04_06, kde_lrg_06_08, kde_lrg_08_11, kde_qso_08_21]
dataframes_list = [chains_ELGs_z_1p1_1p6, chains_LRGs_z_0p4_0p6, chains_LRGs_z_0p6_0p8,
                   chains_LRGs_z_0p8_1p1, chains_QSOs_z_0p8_2p1] 

#total = 0.0 
Zs_list_max = []

xmin = 0.95
xmax = 1.05
ymin = -0.05
ymax = 0.05
x = np.linspace(xmin, xmax, 150)
y = np.linspace(ymin, ymax, 150)
X, Y = np.meshgrid(x, y) # get a 2D grid of points 

positions = np.vstack([X.ravel(), Y.ravel()]) # unravel the points to have a 2 1D arrays 

for i, k in enumerate(kdes_list): 
    
    # use those points to interpolate
    values = np.vstack([dataframes_list[i][r'$\alpha$'].to_numpy(), dataframes_list[i][r'$\epsilon$'].to_numpy()])

    # take the interpolated points and shape the output to be 2D 
    z0 = kdes_list[i](positions)
    Z = np.flip(np.reshape(z0, X.shape), axis=0)
    
    Zs_list_max.append(np.max(z0))
    #total = total + Z

    fig, ax = plt.subplots()

#     ax.plot(dataframes_list[i][r'$\alpha$'].to_numpy(), dataframes_list[i][r'$\epsilon$'].to_numpy(), 
#     'k.', markersize=0.5, alpha = 0.25)
    
    maxxarg, maxyarg = np.unravel_index(np.argmax(np.flip(Z, axis=0).T), shape=X.shape)
    print(maxxarg, maxyarg, x[maxxarg], y[maxyarg])
    ax.scatter(x[maxxarg], y[maxyarg], 
            color='red', s=30)

    ax.imshow(Z, cmap=plt.cm.gist_earth_r,
    extent=[xmin, xmax, ymin, ymax])

    norm = mpl.colors.Normalize(vmin=np.min(Z.reshape(-1)), vmax=np.max(Z.reshape(-1)))

    sm = plt.cm.ScalarMappable(cmap=plt.cm.gist_earth_r, norm=norm)
    plt.colorbar(sm, ax=ax)

    ax.set_xlim([xmin, xmax])

    ax.set_ylim([ymin, ymax])

    plt.show()

In [ ]:

# fig, ax = plt.subplots()
# ax.imshow(total, cmap=plt.cm.gist_earth_r, extent=[xmin, xmax, ymin, ymax])
# ax.vlines(1.00, -0.02, 0.02, linestyle='--', color='k')
# ax.hlines(0.00, 0.98, 1.02, linestyle='--', color='k')

# maxxarg, maxyarg = np.unravel_index(np.argmax(np.flip(total, axis=0).T), shape=X.shape)
    
# ax.scatter(x[maxxarg], y[maxyarg], color='red', s=30)

# ax.set_xlabel(r'$\alpha$')
# ax.set_ylabel(r'$\epsilon$')
# ax.set_xlim([0.985, 1.015])
# ax.set_ylim([-0.015, 0.015])
# norm = mpl.colors.Normalize(vmin=np.min(total), vmax=np.max(total))
# sm = plt.cm.ScalarMappable(cmap = plt.cm.gist_earth_r, norm=norm)
# plt.colorbar(sm, ax=ax, label='Gaussian KDE')
# plt.show()

In [ ]:
# now we have a likelihood, and we need to use MCMC to generate the likelihoods with error bars 
import emcee 

def log_prob(x):
    
    # x is a vector with alpha, epsilon x 5 for each dataset - in a given order 
    # get the likelihood from each KDE 
    elg = kdes_list[0]([x[0], x[1]])[0] 
    if elg <= 0.0 or abs(x[0])-1.0 >= 0.2 or abs(x[1]) >= 0.2 or abs(elg) == np.inf:
        elg = -np.inf 
    else:
        elg = np.log(elg)
        
    lrg1 = kdes_list[1]([x[2], x[3]])[0] 
    if lrg1 <= 0.0 or abs(x[2])-1.0 >= 0.2 or abs(x[3]) >= 0.2 or abs(lrg1) == np.inf:
        lrg1 = -np.inf 
    else:
        lrg1 = np.log(lrg1)
        
    lrg2 = kdes_list[2]([x[4], x[5]])[0] 
    if lrg2 <= 0.0 or abs(x[4])-1.0 >= 0.2 or abs(x[5]) >= 0.2 or abs(lrg2) == np.inf: 
        lrg2 = -np.inf 
    else:
        lrg2 = np.log(lrg2)
        
    lrg3 = kdes_list[3]([x[6], x[7]])[0] 
    if lrg3 <= 0.0 or abs(x[6])-1.0 >= 0.2 or abs(x[7]) >= 0.2 or abs(lrg3) == np.inf: 
        lrg3 = -np.inf 
    else:
        lrg3 = np.log(lrg3)
    
    qso = kdes_list[4]([x[8], x[9]])[0] 
    if qso <= 0.0 or abs(x[8])-1.0 >= 0.2 or abs(x[9]) >= 0.2 or abs(qso) == np.inf: 
        qso = -np.inf 
    else:
        qso = np.log(qso)
        
    if abs(lrg1) == np.inf or abs(lrg2) == np.inf or abs(lrg3) == np.inf or abs(elg) == np.inf or abs(qso) == np.inf:
        logl = -np.inf 
    else: 
        logl = elg + lrg1 + lrg2 + lrg3 + qso
    
    #print(elg, lrg1, lrg2, lrg3, qso, logl, x[8], x[9])
    
    return logl 

ndim = 10
np.random.seed(42)
nwalkers = 32                                                                                          
p0 = np.array([np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers),  
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
                 ]).T

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)

state = sampler.run_mcmc(p0, 1000)
sampler.reset()

state = sampler.run_mcmc(state, 1000)



In [ ]:
state = sampler.run_mcmc(state, 5000) # run again optionally 

In [ ]:
print(sampler.get_chain(flat=True).shape)

In [ ]:
# import corner 

# labels = [r'$\alpha_{\mathrm{ELG}}$', r'$\epsilon_{\mathrm{ELG}}$',
#           r'$\alpha_{\mathrm{LRG,1}}$', r'$\epsilon_{\mathrm{LRG,1}}$',
#           r'$\alpha_{\mathrm{LRG,2}}$', r'$\epsilon_{\mathrm{LRG,2}}$',
#           r'$\alpha_{\mathrm{LRG,3}}$', r'$\epsilon_{\mathrm{LRG,3}}$',
#           r'$\alpha_{\mathrm{QSO}}$', r'$\epsilon_{\mathrm{QSO}}$',
#          ]

# #print(sampler.get_chain(flat=True).shape)
# figure = corner.corner(sampler.get_chain(flat=True), labels=labels)


labels = [r'$\alpha_{\mathrm{ELG}}$', 
          r'$\epsilon_{\mathrm{ELG}}$',
          r'$\alpha_{\mathrm{LRG,1}}$', 
          r'$\epsilon_{\mathrm{LRG,1}}$',
          r'$\alpha_{\mathrm{LRG,2}}$', 
          r'$\epsilon_{\mathrm{LRG,2}}$',
          r'$\alpha_{\mathrm{LRG,3}}$', 
          r'$\epsilon_{\mathrm{LRG,3}}$',
          r'$\alpha_{\mathrm{QSO}}$', 
          r'$\epsilon_{\mathrm{QSO}}$', 
          #r'$\beta_{N_{\mathrm{eff}}}$'
         ]

truth = {r'$\alpha_{\mathrm{ELG}}$': 1.0,
          r'$\epsilon_{\mathrm{ELG}}$': 0.0, 
          r'$\alpha_{\mathrm{LRG,1}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,1}}$': 0.0,
          r'$\alpha_{\mathrm{LRG,2}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,2}}$': 0.0,
          r'$\alpha_{\mathrm{LRG,3}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,3}}$': 0.0,
          r'$\alpha_{\mathrm{QSO}}$': 1.0,
          r'$\epsilon_{\mathrm{QSO}}$': 0.0,
          #r'$\beta_{N_{\mathrm{eff}}}$': 1.0
        } 
from chainconsumer import ChainConsumer 
from chainconsumer import Chain, Truth
c = ChainConsumer() 

chains_flat = sampler.get_chain(flat=True)

plt.rcParams['font.size'] = 12
df = pd.DataFrame({labels[i]: chains_flat[:,i] for i in np.arange(len(chains_flat[0,:]))})
#print(df)

c.add_chain(Chain(samples=df, name='chains'))
c.add_truth(Truth(location=truth))
c.plotter.plot(
    #truth=truth,
    #parameters=labels,
    #legend=False, 
    #show_contour_labels=True
    )


import pickle 

path = '/global/homes/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/'
with open(path+"datasecondgenmockscombinedchains_anisotropicstandard.txt", 'wb') as f: 
    pickle.dump(sampler,f) 

# with open(path+"datasecondgenmockscombinedchains.txt", 'rb') as f: 
#     sampler = pickle.load(f) 


# now make the same KDEs for the data with beta included in the fit and use it to find the likelihood for the parameters 

In [ ]:
# read in the chains for the fits
# lets first just look at alpha and epsilon
# neglecting low z ELGs as they are only weakly constraining and likewise QSO cross-correlations as they are likely small

# using post-recon mocks for all except QSOs - too few QSOs to have good reconstruction 
path = '/global/u1/a/abbew25/files_secondgen_chains/'

listt = [r'$\alpha$', r'$\epsilon$', r'$\beta_{\phi(N_{\mathrm{eff}})}$', 'weights']
#chains_ELGs_z_0p8_1p1 = pd.read_csv(path + 'ELG_08_11_recon' + '.csv')[list]
chains_ELGs_z_1p1_1p6 = pd.read_csv(path + 'ELG_11_16_recon_phaseshift' + '.csv')[listt]
chains_LRGs_z_0p4_0p6 = pd.read_csv(path + 'LRG_04_06_recon_phaseshift' + '.csv')[listt]
chains_LRGs_z_0p6_0p8 = pd.read_csv(path + 'LRG_06_08_recon_phaseshift' + '.csv')[listt]
chains_LRGs_z_0p8_1p1 = pd.read_csv(path + 'LRG_08_11_recon_phaseshift' + '.csv')[listt]
chains_QSOs_z_0p8_2p1 = pd.read_csv(path + 'QSO_08_21_prerecon_phaseshift' + '.csv')[listt]


In [ ]:
from scipy.stats import gaussian_kde

kde_elg_11_16 = gaussian_kde(np.vstack([
    chains_ELGs_z_1p1_1p6[r'$\alpha$'].to_numpy(),
    chains_ELGs_z_1p1_1p6[r'$\epsilon$'].to_numpy(), 
    chains_ELGs_z_1p1_1p6[r'$\beta_{\phi(N_{\mathrm{eff}})}$'].to_numpy(), 
    ]), 
    weights=chains_ELGs_z_1p1_1p6['weights'].to_numpy())

kde_lrg_04_06 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p4_0p6[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p4_0p6[r'$\epsilon$'].to_numpy(),
    chains_LRGs_z_0p4_0p6[r'$\beta_{\phi(N_{\mathrm{eff}})}$'].to_numpy(),   
    ]), 
    weights=chains_LRGs_z_0p4_0p6['weights'].to_numpy())

kde_lrg_06_08 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p6_0p8[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p6_0p8[r'$\epsilon$'].to_numpy(),
    chains_LRGs_z_0p6_0p8[r'$\beta_{\phi(N_{\mathrm{eff}})}$'].to_numpy(), 
    ]), 
    weights=chains_LRGs_z_0p6_0p8['weights'].to_numpy())

kde_lrg_08_11 = gaussian_kde(np.vstack([
    chains_LRGs_z_0p8_1p1[r'$\alpha$'].to_numpy(),
    chains_LRGs_z_0p8_1p1[r'$\epsilon$'].to_numpy(),
    chains_LRGs_z_0p8_1p1[r'$\beta_{\phi(N_{\mathrm{eff}})}$'].to_numpy(), 
    ]), 
    weights=chains_LRGs_z_0p8_1p1['weights'].to_numpy())

kde_qso_08_21 = gaussian_kde(np.vstack([
    chains_QSOs_z_0p8_2p1[r'$\alpha$'].to_numpy(),
    chains_QSOs_z_0p8_2p1[r'$\epsilon$'].to_numpy(),
    chains_QSOs_z_0p8_2p1[r'$\beta_{\phi(N_{\mathrm{eff}})}$'].to_numpy(), 
    ]), 
    weights=chains_QSOs_z_0p8_2p1['weights'].to_numpy())

In [ ]:
# ok, now lets add the 'likelihoods' from the functions together by taking the log of all of them then summing them 

kdes_list = [kde_elg_11_16, kde_lrg_04_06, kde_lrg_06_08, kde_lrg_08_11, kde_qso_08_21]
dataframes_list = [chains_ELGs_z_1p1_1p6, chains_LRGs_z_0p4_0p6, chains_LRGs_z_0p6_0p8,
                   chains_LRGs_z_0p8_1p1, chains_QSOs_z_0p8_2p1] 


In [ ]:
# now we have a likelihood, and we need to use MCMC to generate the likelihoods with error bars 
import emcee 

def log_prob(x):
    
    # x is a vector with alpha, epsilon x 5 for each dataset - in a given order 
    # get the likelihood from each KDE 
    elg = kdes_list[0]([x[0], x[1], x[10]])[0] 
    if elg <= 0.0 or abs(x[0])-1.0 >= 0.2 or abs(x[1]) >= 0.2 or abs(elg) == np.inf:
        elg = -np.inf 
    else:
        elg = np.log(elg)
        
    lrg1 = kdes_list[1]([x[2], x[3], x[10]])[0] 
    if lrg1 <= 0.0 or abs(x[2])-1.0 >= 0.2 or abs(x[3]) >= 0.2 or abs(lrg1) == np.inf: 
        lrg1 = -np.inf 
    else:
        lrg1 = np.log(lrg1)
        
    lrg2 = kdes_list[2]([x[4], x[5], x[10]])[0] 
    if lrg2 <= 0.0 or abs(x[4])-1.0 >= 0.2 or abs(x[5]) >= 0.2 or abs(lrg2) == np.inf: 
        lrg2 = -np.inf 
    else:
        lrg2 = np.log(lrg2)
        
    lrg3 = kdes_list[3]([x[6], x[7], x[10]])[0] 
    if lrg3 <= 0.0 or abs(x[6])-1.0 >= 0.2 or abs(x[7]) >= 0.2 or abs(lrg3) == np.inf: 
        lrg3 = -np.inf 
    else:
        lrg3 = np.log(lrg3)
    
    qso = kdes_list[4]([x[8], x[9], x[10]])[0] 
    if qso <= 0.0 or abs(x[8])-1.0 >= 0.2 or abs(x[9]) >= 0.2 or abs(qso) == np.inf: 
        qso = -np.inf 
    else:
        qso = np.log(qso)
        
    if abs(lrg1) == np.inf or abs(lrg2) == np.inf or abs(lrg3) == np.inf or abs(elg) == np.inf or abs(qso) == np.inf:
        logl = -np.inf 
    elif x[10] > 2.4 or x[10] < -1.0:
        logl = -np.inf
    else: 
        logl = elg + lrg1 + lrg2 + lrg3 + qso
    
    #if np.iscomplex(logl):
    #print(logl, x)
        
    return logl 

ndim = 11
np.random.seed(42)
nwalkers = 32                                                                                          
p0 = np.array([np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers),  np.random.uniform(-0.01, 0.01, nwalkers), 
               np.random.uniform(0.99, 1.01, nwalkers)
                 ]).T

# We'll track how the average autocorrelation time estimate changes

max_n = 10000
index = 0
autocorr = np.empty(max_n)

# This will be useful to testing convergence
old_tau = np.inf

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)

# state = sampler.run_mcmc(p0, 1000)
# sampler.reset()

# state = sampler.run_mcmc(state, 1000)

In [ ]:
# state = sampler.run_mcmc(state, 10000) # run again optionally 

# Now we'll sample for up to max_n steps
for sample in sampler.sample(p0, iterations=max_n, progress=True):
    # Only check convergence every 100 steps
    if sampler.iteration % 100:
        continue

    # Compute the autocorrelation time so far
    # Using tol=0 means that we'll always get an estimate even
    # if it isn't trustworthy
    tau = sampler.get_autocorr_time(tol=0)
    autocorr[index] = np.mean(tau)
    index += 1

    # Check convergence
    converged = np.all(tau * 100 < sampler.iteration)
    converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
    if converged:
        break
    old_tau = tau
    


In [ ]:
# print(sampler.get_chain(flat=True).shape)

# import pickle 

# path = '/global/homes/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/'
# # with open(path+"datasecondgenmockscombinedchains.txt", 'wb') as f: 
# #     pickle.dump(sampler,f) 

with open(path+"datasecondgenmockscombinedchains.txt", 'rb') as f: 
    sampler = pickle.load(f) 



In [ ]:
# import corner 
# plt.rcParams['font.size'] = 15
# labels = [r'$\alpha_{\mathrm{ELG}}$', r'$\epsilon_{\mathrm{ELG}}$',
#           r'$\alpha_{\mathrm{LRG,1}}$', r'$\epsilon_{\mathrm{LRG,1}}$',
#           r'$\alpha_{\mathrm{LRG,2}}$', r'$\epsilon_{\mathrm{LRG,2}}$',
#           r'$\alpha_{\mathrm{LRG,3}}$', r'$\epsilon_{\mathrm{LRG,3}}$',
#           r'$\alpha_{\mathrm{QSO}}$', r'$\epsilon_{\mathrm{QSO}}$', r'$\beta_{N_{\mathrm{eff}}}$'
#          ]

# #print(sampler.get_chain(flat=True).shape)
# figure = corner.corner(sampler.get_chain(flat=True, discard=2000), labels=labels, #quantiles=[0.5-0.341, 0.5, 0.5+0.341], 
#                        show_titles=True, title_fmt = '.3f',
# truths=[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1., 0., 1., 0., 1.], truth_color='blue')


#show the samplers
# fig, axes = plt.subplots(11)#, figsize=(10, 7), sharex=True)
# samples = sampler.get_chain()

labels = [r'$\alpha_{\mathrm{ELG}}$', 
          r'$\epsilon_{\mathrm{ELG}}$',
          r'$\alpha_{\mathrm{LRG,1}}$', 
          r'$\epsilon_{\mathrm{LRG,1}}$',
          r'$\alpha_{\mathrm{LRG,2}}$', 
          r'$\epsilon_{\mathrm{LRG,2}}$',
          r'$\alpha_{\mathrm{LRG,3}}$', 
          r'$\epsilon_{\mathrm{LRG,3}}$',
          r'$\alpha_{\mathrm{QSO}}$', 
          r'$\epsilon_{\mathrm{QSO}}$', 
          r'$\beta_{N_{\mathrm{eff}}}$'
         ]

truth = {r'$\alpha_{\mathrm{ELG}}$': 1.0,
          r'$\epsilon_{\mathrm{ELG}}$': 0.0, 
          r'$\alpha_{\mathrm{LRG,1}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,1}}$': 0.0,
          r'$\alpha_{\mathrm{LRG,2}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,2}}$': 0.0,
          r'$\alpha_{\mathrm{LRG,3}}$': 1.0,
          r'$\epsilon_{\mathrm{LRG,3}}$': 0.0,
          r'$\alpha_{\mathrm{QSO}}$': 1.0,
          r'$\epsilon_{\mathrm{QSO}}$': 0.0,
          r'$\beta_{N_{\mathrm{eff}}}$': 1.0
        } 
from chainconsumer import ChainConsumer 
from chainconsumer import Chain
c = ChainConsumer() 

chains_flat = sampler.get_chain(flat=True, discard=5000)



df = pd.DataFrame({labels[i]: chains_flat[:,i] for i in np.arange(len(chains_flat[0,:]))})

c.add_chain(Chain(samples=df, name='chains'))
c.add_truth(Truth(location=truth))
c.plotter.plot(
    #truth=truth,
    #parameters=labels,
    #legend=False, 
    #show_contour_labels=True
    )

# for i in range(ndim):
#     ax = axes[i]
#     ax.plot(samples[:, :, i], "k", alpha=0.3)
#     #ax.set_xlim(0, len(samples))
#     ax.set_ylabel(labels[i])
#     ax.yaxis.set_label_coords(-0.1, 0.5)

# axes[-1].set_xlabel("step number");
#discards the firsts
# flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)

In [ ]:
clear